# Group Chat Analysis System
Merupakan suatu sistem untuk menganalisa suatu pesan di dalam group apakah termasuk pesan yang baik, buruk, benar, atau hoax. Dengan analisa tersebut, nantinya dapat disimpulkan bahwa semakin tinggi dan banyak pesan yang teranalisa buruk atau hoax maka group tersebut dapat dipastikan adalah group yang tidak benar. Dan sebaliknya, jika di dalam grup tersebut banyak pesan baik dan informasi yang disampaikan adalah benar, maka dapat disimpulkan bahwa group tersebut adalah group yang baik.

Sistem ini disusun oleh

- Fitri Amalia Langgundi
- Khoerul Umam
- Muhammad Ikhsan Hadi
- Rozak Ilham Aditya

Dalam mata kuliah : Sistem Pakar

### Group Chat Analysis 
Inisalisasi library yang dibutuhkan. Beberapa library yang digunakan diantaranya, library untuk memfilter pesan, library untuk mengolah link, library untuk mengolah dataset, library untuk menentukan tingkat karakteristik pesan, dan library lain akan ditambahkan seiring dengan bertambahnya kompleksitas sistem

In [53]:
import re
import nltk
import string
import tldextract
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from difflib import SequenceMatcher as sm

#### Inisialisai Class
Class yang digunakan diberi nama class Pakar. Di dalamnya terdapat berbagai fungsi yang akan memproses data pesan sehingga dapat diketahui karakteristik pesan tersebut. Saat ini sistem hanya dapat memproses satu per satu pesan.

In [54]:
class Pakar:
    def __init__ (self):
        self.text = ''
        self.text_cl = ''
        self.link = []
        self.domaindataset = pd.read_csv('domain.csv', skiprows=0)
        self.badurl = pd.read_csv('urldata.csv', skiprows=0)
        self.result_hoax_arr = []
        self.result_kata_arr = []
        self.result_hoax = 0
        self.result_kata = 0
        self.mean_kotor = []
        self.mean_hoax_cumulative = []
        self.mean_kotor_cumulative = []

    def process(self, text):
        self.text = text
        text_cl = re.sub(r'https?:\/\/\S*', "", text)
        self.text_cl = self.stem_text(text_cl);
        self.checkLink()
        self.checkText(True)
        self.checkText(False)

    def stem_text(self, text):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer() 
        textToClean = text
        cleanText = stemmer.stem(textToClean) 
        cleanText = cleanText.lower()
        cleanText = re.sub(r"\d+", "", cleanText)
        cleanText = cleanText.translate(str.maketrans('','', string.punctuation))
        cleanText = cleanText.strip()
        tokens = nltk.tokenize.word_tokenize(text)
        kataSambung = set(stopwords.words('indonesian'))
        cleanTokens = []
        for te in tokens:
            if te not in kataSambung:
                cleanTokens.append(te)
        result = ' '
        return result.join(cleanTokens)

    def checkLink(self):
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        url = re.findall(regex, self.text)       
        for x in url:
            link = x[0]
            domain = tldextract.extract(x[0])
            suffix = domain.suffix

            if (domain.suffix == ''):
                suffix = domain.domain
            
            checkDomain = self.domaindataset.loc[self.domaindataset['Domain'] == '.' + suffix]
            cleanUrl = re.sub(r'https?:\/\/', "", x[0])
            checkUrl = self.badurl.loc[self.badurl['url'] == cleanUrl]

            if checkUrl.empty == False:
                self.link.append([link, checkUrl['label'].values, checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                break
                
            if (link.find('https') != -1):
                if (checkDomain.empty == False):
                    self.link.append([link, 'Aman', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                else:
                    self.link.append([link, 'Mencurigakan', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
            else:
                if (checkDomain.empty == False):
                    self.link.append([link, 'Keamanan Lemah', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                else:
                    self.link.append([link, 'Bahaya', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])

    def checkText(self, hoax = True):
        mean_kotor = []
        mean_hoax = []
        if hoax:
            dataset = pd.read_csv('D:\Project\Self\Pakar\hoax.csv', header=None)
            for index, row in dataset.iterrows():
                mean_hoax.append(sm(None, self.text_cl, row[0]).ratio())
            self.result_hoax = np.amax(np.array(mean_hoax))
            self.result_hoax_arr = mean_hoax
            self.mean_hoax_cumulative.append(mean_hoax)
        else:
            dataset = pd.read_csv('D:\Project\Self\Pakar\katakotor.csv', header=None)
            for word in self.text_cl.split():
                for index, row in dataset.iterrows():
                    mean_kotor.append(sm(None, word, row[0]).ratio())
            self.result_kata = np.amax(np.array(mean_kotor))
            self.result_kata_arr = mean_kotor
            self.mean_kotor_cumulative.append(mean_kotor)
    
    def remove_emoji(self, text):
        regrex_pattern = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def get_domain_dt(self):
        return self.domaindataset
    
    def get_result(self):
        return self.result

    def get_text(self):
        print("Original : \n")
        print(self.text)
        print("\n\nCleaned : \n")
        print(self.text_cl)

    def get_data(self, data = ''):
        if data == 'hoax':
            return self.result_hoax_arr
        else:
            return self.result_kata_arr

    def print_plot(self, plot = ''):
        if plot == 'hoax':
            pt.plot(self.result_hoax_arr)
        else:
            pt.plot(self.result_kata_arr)

    def show_result(self):
        print('Tingkat ke-hoax-an pesan: ', round(self.result_hoax, 3))
        print('Tingkat ke-kotoran-an pesan: ', round(self.result_kata, 3), '\n')
        if (self.link != None):
            for x in self.link:
                print('Link: ', x[0], ' terindikasi ', x[1])
                print('Domain: ', x[2])
                print('Tipe Link: ', x[3])
                print('Keterangan: ', x[4], '\n')

    def get_all_results(self, plot = ''):
        if plot == 'hoax':
            pt.plot(self.mean_hoax_cumulative)
        else:
            pt.plot(self.mean_kotor_cumulative)




### Testing
Testing digunakan per pesan. Dilakukan seperti di bawah ini, pesan akan diketahui seberapa dekat karakteristik dengan dataset yang digunakan, termasuk jika ada link di dalamnya, akan di ketahui link mana yang berbahaya dan link mana yang aman.

In [ ]:
t = Pakar()
t.process('Bagi yang di sekitar Unnes, khususnya jln taman siswa. Berita kehilangan Dompet warna pink Di dlmnya ada KTP, NPWP,  uang tunai, ATM a.n Yulia Anjani.Brgkli ada yg menemukan, bs hub 089 683 743 789 Mohon bantuannya, Akan di beri imbalan bagi yg menemukan')

In [ ]:
t.show_result()

In [ ]:
t.print_plot('hoax')

In [ ]:
hoaxs = pd.DataFrame(np.array(t.get_data('hoax')))
hoaxs.describe()

In [ ]:
t.get_text()

In [55]:
hoaxDetection = Pakar()
datasetChat = pd.read_csv('D:\Project\Self\Pakar\datachat.csv', header=None, skiprows=1)
cleanDataChat = []
for index, row in datasetChat.iterrows():
    cleanEmoji = hoaxDetection.remove_emoji(row[0])
    if cleanEmoji != '' and cleanEmoji != ' ' and cleanEmoji is not None:
        hoaxDetection.process(cleanEmoji)
hoaxDetection.get_all_result()


ValueError: zero-size array to reduction operation maximum which has no identity